In [1]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

In [10]:
base_folder = Path('./result/readability-indexes')

scaler = MinMaxScaler()

generated_simplified = pd.read_csv(base_folder / 'generated-simplified.csv')
reference_simplified  = pd.read_csv(base_folder / 'reference-simplified.csv')
reference_complete = pd.read_csv(base_folder / 'reference-complete.csv')

In [12]:
generated_simplified["name"] = generated_simplified["name"].str.replace("_stripped.predicted","_simplificado")
reference_simplified["name"] = reference_simplified["name"].str.replace("_stripped.predicted","")
reference_complete["name"] = reference_complete["name"].str.replace("_stripped.predicted","")


In [14]:
generated_simplified

,id,name,year,model,generated_with,number_of_sentences,number_of_tokens,number_of_types,number_of_characters,number_of_syllables,...,hapax_legomena_count,max_sentence_length,flesch_ease,flesch_kincaid,ari,gunning_fog,coleman_liau,gulpease,average_sentence_length,average_syllables_word
0,1,2022_ufc_inova_1_simplificado,2022,port_parser,mistralai/ministral-8b,367,4140,1133,25462,9623.0,...,610,62,46.911598,10.234755,13.254599,8.014960,17.349710,54.091787,11.280654,2.324396
1,1,2023_ufc_inova_1_simplificado,2023,port_parser,mistralai/ministral-8b,226,3080,949,19026,7158.0,...,522,91,44.496679,11.076065,14.411915,9.120733,17.816364,49.240260,13.628319,2.324026
2,3,2021_ufc_inova_3_simplificado,2021,port_parser,mistralai/ministral-8b,229,3570,1151,21449,8218.0,...,665,64,44.045723,11.552619,14.496756,9.980601,17.096807,48.162465,15.589520,2.301961
3,1,2021_ufc_inova_1_simplificado,2021,port_parser,mistralai/ministral-8b,504,6694,998,41555,15810.0,...,431,166,42.136192,11.344321,14.399839,7.643401,17.940992,49.509411,13.281746,2.361817
4,4,2025_ufc_inova_4_simplificado,2025,port_parser,mistralai/ministral-8b,253,3062,936,19048,7232.0,...,454,74,43.359563,10.920288,13.940763,8.437214,17.857022,51.580013,12.102767,2.361855
5,1,2025_ufc_inova_1_simplificado,2025,port_parser,mistralai/ministral-8b,172,3178,956,19158,7283.0,...,489,85,41.782298,12.485234,15.860038,11.379281,17.416362,44.953430,18.476744,2.291693
6,2,2021_ufc_inova_2_simplificado,2021,port_parser,mistralai/ministral-8b,78,1105,412,7173,2435.0,...,252,128,52.606033,10.017647,16.093786,9.348906,19.571222,45.262443,14.166667,2.203620
7,1,2022_ufc_inova_1_simplificado,2022,port_parser,qwen/qwen3-14b,432,4585,1411,28071,10683.0,...,823,38,47.202822,10.052720,12.832743,7.968736,17.082094,56.042530,10.613426,2.329989
8,1,2023_ufc_inova_1_simplificado,2023,port_parser,qwen/qwen3-14b,286,3274,1133,20103,7564.0,...,676,58,47.751235,10.148486,13.281822,8.464524,17.322602,53.804520,11.447552,2.310324
9,3,2021_ufc_inova_3_simplificado,2021,port_parser,qwen/qwen3-14b,181,2083,892,12977,5006.0,...,615,62,40.996336,11.136934,13.721448,9.023122,17.816995,52.768603,11.508287,2.403265


In [16]:
scale_readability_cols = ['flesch_ease','gulpease'] 
grade_level_readability_cols = ['flesch_kincaid','ari','gunning_fog','coleman_liau']
metrics_cols = scale_readability_cols + grade_level_readability_cols


### Teste T

In [25]:
import numpy as np
import scipy.stats as st
import csv
from typing import Any
import math

confidence = 0.95

def generate_confidence_interval(df:pd.DataFrame,metrics:list[str])->pd.DataFrame:
    confidence_intervals: list[dict[str,Any]] = []
    for metric in metrics:
        per_group = df.groupby("generated_with",dropna=False)[f"{metric}"]
        for row in per_group.agg(["mean","count",st.sem]).iterrows():
            range_min,range_max = st.t.interval(confidence=confidence,df=row[1]["count"]-1,loc=row[1]["mean"],scale=row[1]["sem"])
            confidence_intervals.append({"metric":metric,"model":row[0], "range_min":range_min,"mean": row[1]["mean"], "range_max":range_max})

    return pd.DataFrame(confidence_intervals)

generate_confidence_interval(reference_complete,metrics_cols).to_csv('reference-complete-confidence-intervals.csv')

generate_confidence_interval(reference_simplified,metrics_cols).to_csv('reference-simplified-confidence-intervals.csv')

generate_confidence_interval(generated_simplified,metrics_cols).to_csv('generated-simplified-confidence-intervals.csv')